In [107]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import re
import numpy as np
load_dotenv(find_dotenv()) 


True

In [61]:
api_token = os.getenv("api_token")

## import data

In [60]:
# data from 2022
df2201_rent = pd.read_pickle('../data/df_aggregated.pickle')
df2201_sale = pd.read_excel('../data/sale_data_0122.xlsx')

# data from 2023
df2307_rent = pd.read_pickle('../data/2023MaytoJul_rent.pickle')
df2307_sale = pd.read_pickle('../data/2023MaytoJul_sale.pickle')


In [256]:
df2307_sale.ID.nunique()

24

columns to keep: use square feet, deal price, area, street number, address, floor

In [62]:
df2201_rent.head(1)

,use_sf,build_sf,use_rent_sf,build_rent_sf,deal_date,price,area,title,page,location,...,"('distance_km', 'kwun tong station')","('distance_km', 'quarry bay')","('distance_km', 'shenzhen bay port')","('time_min', 'IFC mall')","('time_min', 'West Kowloon Station')","('time_min', 'causeway bay station')","('time_min', 'hong kong international airport')","('time_min', 'kwun tong station')","('time_min', 'quarry bay')","('time_min', 'shenzhen bay port')"
0,1216.0,1579.0,56.0,43.0,2021-09-22,67500.0,西半山,杏彤苑 中層 B室,1,西半山,...,14.548,11.501,43.805,14.366667,18.75,16.116667,35.45,22.983333,22.416667,43.433333


In [63]:
df2201_sale.head(1)

,use_sf,build_sf,use_rent_sf,build_rent_sf,deal_date,price,area,title,page,location,orientation,estate,floor,room
0,1240.0,NaN,2742.0,NaN,2022-01-11,340.0,大嶼山,澄碧邨 9座 2樓 A室,102,大嶼山,NaN,澄碧邨,室,A


In [64]:
df2307_rent.head(1)

,ID,oldTransactionID,transactionPrice,estateName,buildingName,address,yAxis,xAxis,line1,nArea,nUnitPrice
0,AB1202307R0582,20230729-AC-AB1202307R0582,17500,d,百合苑,般咸道20-22號,o,X,百合苑 中層 A室,431,aA


In [65]:
df2307_sale.head(1)

,ID,oldTransactionID,transactionPrice,estateName,buildingName,address,yAxis,xAxis,line1,nArea,nUnitPrice
0,ABC202307S0212,20230728-AC-ABC202307S0212,6750000,d,金豐大廈,香港仔大道171號,o,al,金豐大廈 高層 G室,an,15237


In [148]:
def getfloorHML(address):
    floor_pattern = r'([低中高]層)|(\d+樓)|([一二三四五六七八九十]+樓)' 
    floor = re.findall(floor_pattern, address)
    try:
        floor2 = "".join(list(floor[0])).strip()
    except:
        floor2 = None
    try:
        digits = int(re.findall(r"\d+", floor2)[0])
    except:
        digits = None
    floor_flg = 'M'
    if floor2:    
        if floor2.find("低層") != -1:
            floor_flg = 'L'
        elif floor2.find("高層") != -1:
            floor_flg = 'H'
        elif floor2.find("中層") != -1:
            floor_flg = 'M'
        elif digits:
            if digits <= 10:
                floor_flg = 'L'
            elif digits > 20 :
                floor_flg = 'H'
            
    return floor_flg

getfloorHML('杏彤苑35樓 B室	')



'H'

In [155]:
# https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA&key=YOUR_API_KEY

def getcoordinates(address):
    url = "https://maps.googleapis.com/maps/api/geocode/json?"

    address_list = [address]
    out = []
    try:
        for j in range(len(address_list)):
            r = requests.get(url + 'address=' + address_list[j] + '&key=' + api_token)
            lat = np.nan
            long = np.nan
            if r.json()['status'] != 'ZERO_RESULTS':
                lat = r.json()['results'][0]['geometry']['location']['lat']
                long = r.json()['results'][0]['geometry']['location']['lng']
            # out_ = {
            #     "latitude": lat,
            #     "longitude": long
            # }
            # out.append(out_)
        return pd.Series([lat, long])
    except:
        return pd.Series([np.nan, np.nan])
# df = pd.DataFrame(out)
# df


In [138]:
import json
from shapely.geometry import shape, Point

def WGS84toHK1980Grid(lat,long):
    if np.isnan(lat) or np.isnan(long):
        hkE = np.nan
        hkN = np.nan
    else:
        url = 'http://www.geodetic.gov.hk/transform/v2/?inSys=wgsgeog&outSys=hkgrid&lat='+str(lat)+'&long='+str(long)
        resp = requests.get(url=url)
        data = resp.json()
        try:
            hkN = data['hkN']
        except:
            hkN = np.nan
        try:
            hkE = data['hkE']
        except:
            hkE = np.nan
    return hkE,hkN
# WGS84toHK1980Grid(22.381242, 113.969121)

with open('../data/2019DistrictCouncilOrdinaryElectionConstituencyBoundaries_GEOJSON/DCCA2019.json', encoding='utf-8') as f:
    js = json.load(f)

properties = []
for feature in js["features"]:
    property_ = (feature["properties"])
    properties.append(property_)

df_properties = pd.DataFrame(properties)

def pointtoarea(lat,long):
    if np.isnan(lat) or np.isnan(long):
        return ''
    # load GeoJSON file containing sectors
    # with open('../data/2019DistrictCouncilOrdinaryElectionConstituencyBoundaries_GEOJSON/DCCA2019.json', encoding='utf-8') as f:
    #     js = json.load(f)

    # Convert each feature to a Polygon object
    polygons = []
    for feature in js["features"]:
        polygon = shape(feature["geometry"])
        polygons.append(polygon)

    # properties = []
    # for feature in js["features"]:
    #     property_ = (feature["properties"])
    #     properties.append(property_)

    # Define the coordinate
    point = Point(WGS84toHK1980Grid(lat, long))
    CACODE = ''
    # Check which polygon the coordinate belongs to
    for i, polygon in enumerate(polygons):
        if polygon.contains(point):
            CACODE = properties[i]['CACODE']
            # ENAME = properties[i]['ENAME']
            # DISTRICT_E = properties[i]['DISTRICT_E']
            # CNAME = properties[i]['CNAME']
            # DISTRICT_T = properties[i]['DISTRICT_T']
            break
    else:
        print("The coordinate is not inside any polygon")
    return CACODE


In [258]:
df_properties.sample(1)

,CACODE,ENAME,CNAME,DISTRICT_T,DISTRICT_E,Shape_Leng,Shape_Area
117,E13,Mong Kok North,旺角北,油尖旺區,Yau Tsim Mong District,2320.603155,212719.303149


In [259]:
df_properties.to_excel('../data/CACODE_properties.xlsx',index=False)

df2201_rent

In [128]:
df2201_rent2 = df2201_rent.copy()
df2201_rent2 = df2201_rent2[['use_sf','build_sf','price','area','title','addr_home','floor']]
df2201_rent2['floor_flg'] = df2201_rent2["title"].apply(getfloorHML)
df2201_rent2['use_sf'] = np.where(df2201_rent2['use_sf'].isna(),df2201_rent2['build_sf'],df2201_rent2['use_sf'])
df2201_rent2['addr_home'] = np.where(df2201_rent2['addr_home'].isnull(),df2201_rent2['title'],df2201_rent2['addr_home'])
df2201_rent2[["lat", "long"]] = df2201_rent2["addr_home"].apply(getcoordinates)

df2201_rent2.head(1)

,use_sf,build_sf,price,area,title,addr_home,floor,floor_flg,lat,long
0,1216.0,1579.0,67500.0,西半山,杏彤苑 中層 B室,"['Hatton Place, 1A Po Shan Rd, Mid-Levels, Hon...",中,M,22.279991,114.13896


In [139]:
df2201_rent2["CACODE"] = df2201_rent2.apply(lambda x: pointtoarea(x['lat'],x['long']), axis = 1)


c:\Python311\Lib\site-packages\shapely\predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate i

In [142]:
print(df2201_rent2['use_sf'].isna().sum())
print(df2201_rent2['price'].isna().sum())
print(df2201_rent2['floor_flg'].isna().sum())
print(df2201_rent2['CACODE'].isna().sum())


61
0
0
0


In [143]:
df2201_rent2.to_pickle('../data/df2201_rent.pickle')

In [154]:
df2201_rent2.sample(2)

,use_sf,build_sf,price,area,title,addr_home,floor,floor_flg,lat,long,CACODE
1391,447.0,NaN,23000.0,灣仔,囍匯 2期 1座 中層 E室,"[""200 Queen's Rd E, Wan Chai, Hong Kong""]",中,M,22.275005,114.172064,B12
7837,397.0,NaN,18500.0,旺角,SKYPARK 高層 A2室,SKYPARK 高層 A2室,高,H,NaN,NaN,


df2201_sale

In [156]:
df2201_sale2 = df2201_sale.copy()
df2201_sale2['floor_flg'] = df2201_sale2["title"].apply(getfloorHML)
df2201_sale2['use_sf'] = np.where(df2201_sale2['use_sf'].isna(),df2201_sale2['build_sf'],df2201_sale2['use_sf'])
df2201_sale2['use_rent_sf'] = np.where(df2201_sale2['use_rent_sf'].isna(),df2201_sale2['build_rent_sf'],df2201_sale2['use_rent_sf']) # price per square feet
df2201_sale2['price'] = df2201_sale2['price'].fillna(0)*10000
df2201_sale2['price'] = np.where(df2201_sale2['price'].isna(), df2201_sale2['use_rent_sf'].fillna(0)*df2201_sale2['use_sf'].fillna(0), df2201_sale2['price'])
df2201_sale2['estate'] = np.where(df2201_sale2['estate'].isna(),df2201_sale2['title'],df2201_sale2['estate']) 
df2201_sale2[["lat", "long"]] = df2201_sale2["estate"].apply(getcoordinates)
df2201_sale2["CACODE"] = df2201_sale2.apply(lambda x: pointtoarea(x['lat'],x['long']), axis = 1)
df2201_sale2 = df2201_sale2[['use_sf','build_sf','price','area','title','estate','floor','floor_flg','lat','long','CACODE']]

print(df2201_sale2['use_sf'].isna().sum())
print(df2201_sale2['price'].isna().sum())
print(df2201_sale2['floor_flg'].isna().sum())
print(df2201_sale2['CACODE'].isna().sum())

c:\Python311\Lib\site-packages\shapely\predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate is not inside any polygon
The coordinate i

In [157]:
df2201_sale2.to_pickle('../data/df2201_sale2.pickle')
df2201_sale2.head(1)

,use_sf,build_sf,price,area,title,estate,floor,floor_flg,lat,long,CACODE
0,1240.0,NaN,340000000.0,大嶼山,澄碧邨 9座 2樓 A室,澄碧邨,室,L,22.214953,113.989682,T01


agg to area level


In [ ]:
df2201_rent2['estate'] = df2201_rent2["title"].str.split(" ").str.get(0)
df2201_rent2['rent_per_sqft'] = df2201_rent2['price']/df2201_rent2['use_sf']

df2201_rent_area = pd.pivot_table(df2201_rent2, 
               values=["use_sf","rent_per_sqft","price","title"], 
            #    index="City", 
               index=["CACODE","estate","floor_flg"], 
               aggfunc={"use_sf":"median","rent_per_sqft":"mean","price":"median","title":"count"}).reset_index()

df2201_sale2['sale_per_sqft'] = df2201_sale2['price']/df2201_sale2['use_sf']

df2201_sale_area = pd.pivot_table(df2201_sale2, 
               values=["use_sf","sale_per_sqft","price","title"], 
            #    index="City", 
               index=["CACODE","estate","floor_flg"], 
               aggfunc={"use_sf":"median","sale_per_sqft":"mean","price":"median","title":"count"}).reset_index()

# df2201_sale_area.sample(1)


df2201_area_agg = pd.merge(df2201_rent_area, df2201_sale_area,how='inner',on = ['CACODE','estate','floor_flg'])
df2201_area_agg['price_to_rent_ratio'] = df2201_area_agg['price_y']/(df2201_area_agg['price_x']*12)
print(len(df2201_area_agg)) # 851 left


# get mean lat and long for CACODE
mean_coor_sale = pd.pivot_table(df2201_sale2, 
               values=["lat","long"], 
               index=["CACODE","estate","floor_flg"], 
               aggfunc="mean").reset_index()
df2201_area_agg = pd.merge(df2201_area_agg,mean_coor_sale,how='left',on = ["CACODE","estate","floor_flg"])
df2201_area_agg.sample(1)
df2201_area_agg = df2201_area_agg.rename(columns={"price_x": "price_rent", "use_sf_x": "use_sf_rent", "price_y": "price_sale",
                                                  "use_sf_y":"use_sf_sale","title_x":"count_rent", "title_y":"count_sale"})

df2201_area_agg = df2201_area_agg.fillna(0)
df2201_area_agg = df2201_area_agg[(df2201_area_agg['lat']>21) & (df2201_area_agg['lat']<23) & (df2201_area_agg['long']<115) & (df2201_area_agg['long']>111)]
max_row_value = df2201_area_agg[["count_rent", "count_sale"]].max(axis=1)
df2201_area_agg["count"] = max_row_value

In [214]:

df2201_area_agg.to_excel('../data/forTableau_202201.xlsx', index=False)

df2307_rent

In [ ]:
# ID	oldTransactionID	transactionPrice	estateName	buildingName	address	yAxis	xAxis	line1	nArea	nUnitPrice

In [199]:
df2307_rent2.head(1)

,ID,oldTransactionID,transactionPrice,estateName,buildingName,address,yAxis,xAxis,line1,nArea,nUnitPrice,floor_flg,use_sf,price,estate,building
0,AB1202307R0582,20230729-AC-AB1202307R0582,17500.0,d,百合苑,般咸道20-22號,o,X,百合苑 中層 A室,431.0,NaN,M,431.0,17500.0,d,百合苑


In [243]:
len(df2307_rent)

10008

In [200]:
df2307_rent2 = df2307_rent.copy()
df2307_rent2['transactionPrice'] = df2307_rent2['transactionPrice'].apply(pd.to_numeric, errors='coerce')
df2307_rent2['nArea'] = df2307_rent2['nArea'].apply(pd.to_numeric, errors='coerce')
df2307_rent2['nUnitPrice'] = df2307_rent2['nUnitPrice'].apply(pd.to_numeric, errors='coerce')

df2307_rent2['floor_flg'] = df2307_rent2["line1"].apply(getfloorHML)
# df2307_rent2['use_sf'] = df2307_rent2['nArea']
df2307_rent2['use_sf'] = np.where(df2307_rent2['nArea'].isna(), df2307_rent2['transactionPrice'].fillna(0)/df2307_rent2['nUnitPrice'].fillna(0), df2307_rent2['nArea'])
df2307_rent2['price'] = np.where(df2307_rent2['transactionPrice'].isna(), df2307_rent2['nUnitPrice'].fillna(0)*df2307_rent2['nArea'].fillna(0), df2307_rent2['transactionPrice'])
df2307_rent2['estate'] = df2307_rent2['estateName']
df2307_rent2['building'] = df2307_rent2['buildingName']
df2307_rent2[["lat", "long"]] = df2307_rent2["address"].apply(getcoordinates)
df2307_rent2["CACODE"] = df2307_rent2.apply(lambda x: pointtoarea(x['lat'],x['long']), axis = 1)
df2307_rent2 = df2307_rent2[['use_sf','price','line1','estate','building','floor_flg','lat','long','CACODE']]


print(df2307_rent2['use_sf'].isna().sum())
print(df2307_rent2['price'].isna().sum())
print(df2307_rent2['floor_flg'].isna().sum())
print(df2307_rent2['CACODE'].isna().sum())

df2307_rent2.to_pickle('../data/df2307_rent2.pickle')
df2307_rent2.head(1)

0
0
0
0


,use_sf,price,line1,estate,building,floor_flg,lat,long,CACODE
0,431.0,17500.0,百合苑 中層 A室,d,百合苑,M,22.283675,114.146049,A13


In [215]:
df2307_sale2 = df2307_sale.copy()
df2307_sale2['transactionPrice'] = df2307_sale2['transactionPrice'].apply(pd.to_numeric, errors='coerce')
df2307_sale2['nArea'] = df2307_sale2['nArea'].apply(pd.to_numeric, errors='coerce')
df2307_sale2['nUnitPrice'] = df2307_sale2['nUnitPrice'].apply(pd.to_numeric, errors='coerce')

df2307_sale2['floor_flg'] = df2307_sale2["line1"].apply(getfloorHML)
# df2307_sale2['use_sf'] = df2307_sale2['nArea']
df2307_sale2['use_sf'] = np.where(df2307_sale2['nArea'].isna(), df2307_sale2['transactionPrice'].fillna(0)/df2307_sale2['nUnitPrice'].fillna(0), df2307_sale2['nArea'])
df2307_sale2['price'] = np.where(df2307_sale2['transactionPrice'].isna(), df2307_sale2['nUnitPrice'].fillna(0)*df2307_sale2['nArea'].fillna(0), df2307_sale2['transactionPrice'])
df2307_sale2['estate'] = df2307_sale2['estateName']
df2307_sale2['building'] = df2307_sale2['buildingName']
df2307_sale2[["lat", "long"]] = df2307_sale2["address"].apply(getcoordinates)
df2307_sale2["CACODE"] = df2307_sale2.apply(lambda x: pointtoarea(x['lat'],x['long']), axis = 1)
df2307_sale2 = df2307_sale2[['use_sf','price','line1','estate','building','floor_flg','lat','long','CACODE']]


print(df2307_sale2['use_sf'].isna().sum())
print(df2307_sale2['price'].isna().sum())
print(df2307_sale2['floor_flg'].isna().sum())
print(df2307_sale2['CACODE'].isna().sum())


df2307_sale2.to_pickle('../data/df2307_sale2.pickle')
df2307_sale2.head(1)

0
0
0
0


,use_sf,price,line1,estate,building,floor_flg,lat,long,CACODE
0,443.000591,6750000.0,金豐大廈 高層 G室,d,金豐大廈,H,22.248375,114.155501,D01


In [219]:
df2307_rent2['price'].describe()

count    10008.000000
mean     18389.583333
std      12852.571130
min          0.000000
25%      12775.000000
50%      16750.000000
75%      22250.000000
max      56800.000000
Name: price, dtype: float64

In [246]:
# df2307_sale2.to_pickle('../data/df2307_sale2.pickle')
# df2307_rent2.to_pickle('../data/df2307_rent2.pickle')


In [251]:
df2307_rent2 = df2307_rent2[df2307_rent2['price']!=0]
df2307_rent2["use_sf"] = df2307_rent2["use_sf"].replace([np.inf, -np.inf], np.nan)
df2307_rent2 = df2307_rent2.dropna()
df2307_rent2['estate'] = df2307_rent2["line1"].str.split(" ").str.get(0)
df2307_rent2['rent_per_sqft'] = df2307_rent2['price']/df2307_rent2['use_sf']


C:\Users\1\AppData\Local\Temp\ipykernel_15004\2111006949.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2307_rent2["use_sf"] = df2307_rent2["use_sf"].replace([np.inf, -np.inf], np.nan)


In [ ]:

df2307_rent_area = pd.pivot_table(df2307_rent2, 
               values=["use_sf","rent_per_sqft","price","line1"], 
            #    index="City", 
               index=["CACODE","estate","floor_flg"], 
               aggfunc={"use_sf":"median","rent_per_sqft":"mean","price":"median","line1":"count"}).reset_index()

df2307_sale2["use_sf"] = df2307_sale2["use_sf"].replace([np.inf, -np.inf], np.nan)
df2307_sale2 = df2307_sale2.dropna()

df2307_sale2['sale_per_sqft'] = df2307_sale2['price']/df2307_sale2['use_sf']
df2307_sale2['estate'] = df2307_sale2["line1"].str.split(" ").str.get(0)

df2307_sale_area = pd.pivot_table(df2307_sale2, 
               values=["use_sf","sale_per_sqft","price","line1"], 
            #    index="City", 
               index=["CACODE","estate","floor_flg"], 
               aggfunc={"use_sf":"median","sale_per_sqft":"mean","price":"median","line1":"count"}).reset_index()

# df2307_sale_area.sample(1)


df2307_area_agg = pd.merge(df2307_rent_area, df2307_sale_area,how='inner',on = ['CACODE','estate','floor_flg'])
df2307_area_agg['price_to_rent_ratio'] = df2307_area_agg['price_y']/(df2307_area_agg['price_x']*12)
print(len(df2307_area_agg)) #  left


# get mean lat and long for CACODE
mean_coor_sale = pd.pivot_table(df2307_sale2, 
               values=["lat","long"], 
               index=["CACODE","estate","floor_flg"], 
               aggfunc="mean").reset_index()
df2307_area_agg = pd.merge(df2307_area_agg,mean_coor_sale,how='left',on = ["CACODE","estate","floor_flg"])
df2307_area_agg = df2307_area_agg.rename(columns={"price_x": "price_rent", "use_sf_x": "use_sf_rent", "price_y": "price_sale",
                                                  "use_sf_y":"use_sf_sale","line1_x":"count_rent", "line1_y":"count_sale"})

df2307_area_agg = df2307_area_agg.fillna(0)
df2307_area_agg = df2307_area_agg[(df2307_area_agg['lat']>21) & (df2307_area_agg['lat']<23) & (df2307_area_agg['long']<115) & (df2307_area_agg['long']>111)]
max_row_value = df2307_area_agg[["count_rent", "count_sale"]].max(axis=1)
df2307_area_agg["count"] = max_row_value

In [264]:
sample_rent_2307 = df2307_rent2.drop_duplicates()
sample_rent_2307.to_excel('../data/sample_rent_2307.xlsx',index=False)

In [263]:
pd.pivot_table(df2307_rent2, 
               values=["use_sf","rent_per_sqft","price","line1"], 
            #    index="City", 
               index=["CACODE","estate","floor_flg"], 
               aggfunc={"use_sf":"median","rent_per_sqft":"mean","price":"median","line1":"count"}).reset_index()


,CACODE,estate,floor_flg,line1,price,rent_per_sqft,use_sf
0,A10,創業中心,L,417,16000.0,42.553191,376.0
1,A13,百合苑,M,417,17500.0,40.603248,431.0
2,D04,深灣軒,M,417,17800.0,36.853002,483.0
3,D11,貝沙灣,M,417,40800.0,54.765101,745.0
4,L12,滿名山,L,417,23000.0,27.380952,840.0
5,L14,恆福花園,M,417,15000.0,24.038462,624.0
6,L25,NOVO,H,417,12800.0,29.024943,441.0
7,P09,天賦海灣,H,417,56800.0,33.023256,1720.0
8,Q21,新都城,H,417,15300.0,42.032967,364.0
9,Q28,海茵莊園,M,417,13500.0,55.102041,245.0


In [242]:
df2307_rent

,ID,oldTransactionID,transactionPrice,estateName,buildingName,address,yAxis,xAxis,line1,nArea,nUnitPrice
0,AB1202307R0582,20230729-AC-AB1202307R0582,17500,d,百合苑,般咸道20-22號,o,X,百合苑 中層 A室,431,aA
1,ABC202307R1079,20230729-AC-ABC202307R1079,22000,aD,2座 (2A),aE,o,aF,港島南岸 1期 晉環 2座 (2A) 中層 E室,584,aa
2,AR1202307R0837,20230729-AC-AR1202307R0837,aL,富景花園,ab,干德道58A-58B號,y,t,富景花園 2座 低層 C室,600,aO
3,BNM202307R0544,20230729-AC-BNM202307R0544,56800,K,ac,科進路5號,z,ad,天賦海灣 1期 3座 高層 B室,1720,33
4,CNS202306R2523,20230729-AC-CNS202306R2523,12000,5期,46座,得寶街2號,z,X,沙田第一城 5期 46座 高層 A室,284,aS
...,...,...,...,...,...,...,...,...,...,...,...
19,SYP202306R2645,20230729-AC-SYP202306R2645,16000,創業中心,A座,德輔道西402-404號,y,ad,創業中心 A座 低層 B室,376,aO
20,TBL202307R0851,20230729-AC-TBL202307R0851,bQ,學士臺,ac,薄扶林道101號,y,X,學士臺 3座 低層 A室,531,bS
21,TSB202307R1239,20230729-AC-TSB202307R1239,34000,海景花園(西),綠楊閣 (35座),太古灣道22號,o,F室,太古城 海景花園(西) 綠楊閣 (35座) 中層 F室,bF,aa
22,UCS202305R1470,20230729-AC-UCS202305R1470,25000,K,5座,美田路1號,z,SD室,名城 1期 5座 高層 SD室,739,bS
